In [1]:
import numpy as np
import pandas as pd

In [2]:
countries = pd.read_csv('src/geonames/countryInfo.txt',header=50,sep='\t')

In [3]:
countries = countries.rename({'#ISO':'ISO'},axis=1)

In [4]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ISO                 251 non-null    object 
 1   ISO3                252 non-null    object 
 2   ISO-Numeric         252 non-null    int64  
 3   fips                249 non-null    object 
 4   Country             252 non-null    object 
 5   Capital             246 non-null    object 
 6   Area(in sq km)      246 non-null    float64
 7   Population          252 non-null    int64  
 8   Continent           210 non-null    object 
 9   tld                 250 non-null    object 
 10  CurrencyCode        251 non-null    object 
 11  CurrencyName        251 non-null    object 
 12  Phone               247 non-null    object 
 13  Postal Code Format  149 non-null    object 
 14  Postal Code Regex   147 non-null    object 
 15  Languages           249 non-null    object 
 16  geonamei

In [5]:
country_alt_names = pd.read_csv('src/country_alt_names.csv')
country_alt_names = country_alt_names.drop('Description',axis=1)

In [6]:
country_alt_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 2 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Alpha-3 code                    198 non-null    object
 1   Other name(s) or older name(s)  201 non-null    object
dtypes: object(2)
memory usage: 3.3+ KB


In [7]:
country_alt_names = country_alt_names.rename({'Alpha-3 code':'ISO3',
                                              'Other name(s) or older name(s)':'alternatenames'},
                                             axis=1)

In [8]:
# country_alt_names['country'] = country_alt_names['country'].str.replace(regex=True,
#                                                                               pat='\(.*?\)',
#                                                                               repl='').str.strip()

country_alt_names['alternatenames'] = country_alt_names['alternatenames'].str.replace(regex=True,
                                                                              pat='\(.*?\)',
                                                                              repl='').str.strip()

country_alt_names['alternatenames'] = country_alt_names['alternatenames'].str.replace(regex=True,
                                                                              pat='\\n',
                                                                              repl='')

In [9]:
xkx_idx = country_alt_names[country_alt_names['alternatenames'].str.contains('Kosovo')].index
country_alt_names['ISO3'].iloc[xkx_idx] = 'XKX'

In [10]:
ctran_to_ctr = countries[['ISO3','Country']].merge(country_alt_names,
                                                          on='ISO3').drop('ISO3',
                                                                          axis=1)

In [11]:
mask = ctran_to_ctr['alternatenames'].notna()
ctran_to_ctr.loc[mask,
                 'alternatenames'] = ctran_to_ctr.loc[mask,
                                                      'alternatenames'].str.cat(ctran_to_ctr.loc[mask,
                                                                                                 'Country'],
                                                                                sep=',')

In [12]:
ctran_to_ctr['alternatenames'] = ctran_to_ctr['alternatenames'].fillna(ctran_to_ctr['Country'])

In [13]:
ctran_to_ctr['alternatenames'] = ctran_to_ctr['alternatenames'].str.replace('\W$','')
ctran_to_ctr['alternatenames'] = ctran_to_ctr['alternatenames'].str.split(',')
ctran_to_ctr = ctran_to_ctr.explode('alternatenames')
ctran_to_ctr['alternatenames'] = ctran_to_ctr['alternatenames'].str.strip()

# ctran_to_ctr = ctran_to_ctr[ctran_mapto_stdname['altNames'] != ctran_to_ctr['Country']]

In [14]:

ctran_to_ctr = ctran_to_ctr.dropna()
# ctran_to_ctr = ctran_to_ctr.set_index('altNames',drop=True)['Country']

In [15]:
# ctran_to_ctr = ctran_to_ctr.drop('Republic of the Congo',errors='ignore')
ctran_to_ctr = ctran_to_ctr[ctran_to_ctr['Country']!='Republic of the Congo']

In [16]:
# ctran_to_ctr = ctran_to_ctr.append(
#     pd.Series(
#         index=['Wales','Scotland','England','Northern Ireland'],
#         data=['United Kingdom','United Kingdom','United Kingdom','United Kingdom']
#     )
# )

In [19]:
ctran_to_ctr = ctran_to_ctr.append(
    pd.DataFrame(
        data=[('United Kingdom','Wales'),
             ('United Kingdom','Scotland'),
             ('United Kingdom','England'),
             ('United Kingdom','Northern Ireland')],
        columns=ctran_to_ctr.columns
    ),
    ignore_index=True
)

In [21]:
ctran_to_ctr = ctran_to_ctr.drop_duplicates('alternatenames')

In [23]:
ctran_to_ctr.to_csv('geo/str_maps/ctran_to_ctr.csv',index=False)